In [11]:
from Functions import *
from Optimizations import *
from utils import *
from Saving import *

%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
import time
import yaml

In [3]:
f = flat_sharp_gaussian
grad_f = grad_flat_sharp_gaussian

x_range = [-25, 25]

In [18]:
A = np.array([[1]])
b = np.array([0])
f = QuadraticFunctionInit(A, b)
grad_f = GradQuadraticFunctionInit(A)

x_range = [-15, 15]

In [26]:
g_params = [[-10, 1, -2], [0, 10, -1], [10, 3, 1], [-20, 1, 3], [20, 1, 3]]
f = gaussian_sum(g_params)
grad_f = grad_gaussian_sum(g_params)

x_range = [-20, 20]

In [43]:
num_particles = 1000
process = {}

process["potential_function"] = {"name": "gaussian", "params": g_params}

process["total_iter"] = 20
process["tau"] = 50
process["x_range"] = x_range

process["start"] = [[np.random.uniform(x_range[0], x_range[1])] for _ in range(num_particles)]
process["gamma"] = lambda t: 2.5
process["temperature"] = lambda t: 1
process["epsilon"] = 0

process["weight_function"] = {"name": "discounted_norm", "params": {"gamma" : 0.999}} 
process["resample_function"] = {"name": "softmax", "params": {"beta": -1}} 

process["domain_enforcer"] = {"name": "hyper_cube_enforcer", "params": {"strength": 0.2}}

all_paths = diffusion_resampling(process)


In [44]:
X = np.linspace(x_range[0], x_range[1], 200)
inp = np.array([X])
Y = f(inp)

all_paths_proc = []

for t in range(len(all_paths)):
    curr_paths = all_paths[t]
    
    all_paths_proc.append([])
    
    for p in range(len(curr_paths)):
        x_curr = np.array(curr_paths[p])
        out = f(x_curr.T)
        all_paths_proc[-1].append(np.concatenate([x_curr, out.reshape(len(out), 1)], axis=1))

all_paths_proc = np.array(all_paths_proc)

In [13]:
# Approximating density with the particles
def V(x, K, particles):
    N = len(particles)
    ret_sum = 0
    for p in particles:
        ret_sum += K(x, p)
    return 1 / float(N) * ret_sum

In [45]:
K = multi_gaussian(np.array([[0.6]]))


saved_path = create_animation_1d_pictures_particles(all_paths_proc, X, Y, graph_details={"p_size": 3, #"density_function": None})
                                                                                      "density_function": 
                                                                                      lambda x, p: V(np.array([x]), K, p)})
with open(saved_path + "/process.yml", "w") as f:
    yaml.dump(process, f, default_flow_style=False)
    
    
create_animation(saved_path, "test.mp4", framerate=15)

In [49]:
with open(ani_path + "/process.yml", "w") as f:
    yaml.dump(process, f, default_flow_style=False)

In [54]:
def remove_png(dir_path):
    import os

    files = os.listdir(dir_path)

    for item in files:
        if item.endswith(".png"):
            os.remove(os.path.join(dir_path, item))

In [55]:
remove_png(ani_path)

### Things to note:

The beta value of the softmax turns out to be very important. Adjusting that value determines how much to value each respective shallow/flat regions. Seems like changing it can give you a nice stationary distribution around certain falt minima. 

Also the tau value is important. Letting the run run for too long will cause the particles to approach more or less the stationary distribution if ran with soley diffusion. 

Maybe penalize where you started from. 

